In [156]:
#Hint
#1.Encode 非數值的feature
#2.Fill median value to Misiing value
#3.處理Data imbalance問題 eg SMOTE
#4.正規化,feature selection
#處理流程:合併train,test ->Encode non numeric value -> fill Missing value ->(feature selection)-> Normalization -> train_test_split
#https://blog.csdn.net/m0_37725003/article/details/81095555
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html
#classification最後 feature selection(講義倒數)
#補缺失值使用 同個class的median(X 不可行) 
#先使用Ensemble方法 不行再用feature selection


In [157]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [158]:
df_train = pd.read_csv('train.csv') #35156
df_test = pd.read_csv('test.csv') #34844

In [159]:
df_test['Weather'] = np.zeros(len(df_test))
train_end_idx = len(df_train)

In [160]:
df = pd.concat([df_train,df_test],ignore_index=True)
df

,Date,Loc,TempLow,TempHigh,Steaming,Sun,WindDir,WindSpeed,DayWindDir,NightWindDir,...,DayHumidity,NightHumidity,DayPressure,NightPressure,DayCloud,NightCloud,DayTemp,NightTemp,Weather,RISK_MM
0,2016/7/19,NaN,19.5,NaN,5.69496,0.500000,ENE,61.0,ENE,ENE,...,86.641375,41.810006,1020.4,1021.9,NaN,7.00000,NaN,19.6,0.0,NaN
1,2010/7/26,Canberra,2.1,15.7,NaN,5.700000,E,26.0,NaN,SE,...,53.275715,76.598048,1023.7,1020.8,4.000000,1.00000,9.0,14.1,0.0,0.0
2,2014/4/2,Woomera,20.9,36.1,5.69496,NaN,S,39.0,S,SW,...,15.471574,52.163254,1017.1,1014.9,4.167266,4.27812,24.2,33.9,0.0,0.0
3,2014/11/19,Tuggeranong,11.5,23.9,5.69496,7.993227,NNE,35.0,W,NaN,...,46.442398,67.664733,1010.2,NaN,4.167266,4.27812,13.8,21.8,1.0,0.2
4,NaN,Hobart,8.2,20.5,3.20000,NaN,N,69.0,N,WNW,...,51.830889,51.770240,NaN,1001.2,7.000000,7.00000,NaN,17.6,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,2011/12/27,NaN,9.6,21.5,8.00000,12.000000,SSW,31.0,ESE,SE,...,39.878487,57.490952,1024.5,1022.8,0.000000,6.00000,14.3,19.0,0.0,0.0
69996,NaN,Canberra,11.6,31.4,5.69496,7.993227,NNE,39.0,NaN,SE,...,NaN,70.743790,1019.1,1016.3,4.167266,4.27812,19.3,NaN,0.0,0.0
69997,2011/1/22,Moree,22.2,34.5,NaN,7.600000,NNE,39.0,NNE,N,...,30.743840,64.412890,NaN,1008.5,7.000000,7.00000,25.2,NaN,0.0,0.0
69998,2010/10/2,Mildura,16.4,43.5,7.00000,12.700000,ENE,43.0,NaN,NaN,...,28.084916,63.565166,1014.2,1010.5,0.000000,4.27812,28.9,NaN,0.0,NaN


In [161]:
#NightWindDir DayWindDir WindDir Loc Date
#drop Loc,Date
df = df.drop(columns=['Date','Loc','NightWindDir','DayWindDir','WindDir']) 

In [162]:
df0=df[df['Weather']==0]
df1=df[df['Weather']==1]
print(df0)
print(np.median(df1['TempLow']))

       TempLow  TempHigh  Steaming        Sun  WindSpeed  DayWindSpeed  \
0         19.5       NaN   5.69496   0.500000       61.0          31.0   
1          2.1      15.7       NaN   5.700000       26.0           9.0   
2         20.9      36.1   5.69496        NaN       39.0          31.0   
4          8.2      20.5   3.20000        NaN       69.0           NaN   
5          9.7      24.1   5.69496        NaN       31.0          11.0   
...        ...       ...       ...        ...        ...           ...   
69995      9.6      21.5   8.00000  12.000000       31.0          17.0   
69996     11.6      31.4   5.69496   7.993227       39.0           7.0   
69997     22.2      34.5       NaN   7.600000       39.0          26.0   
69998     16.4      43.5   7.00000  12.700000       43.0          19.0   
69999      6.9       NaN   6.60000  10.600000        NaN          13.0   

       NightWindSpeed  DayHumidity  NightHumidity  DayPressure  NightPressure  \
0                22.0    86.64

In [163]:
df[df['Weather']==0]

,TempLow,TempHigh,Steaming,Sun,WindSpeed,DayWindSpeed,NightWindSpeed,DayHumidity,NightHumidity,DayPressure,NightPressure,DayCloud,NightCloud,DayTemp,NightTemp,Weather,RISK_MM
0,19.5,NaN,5.69496,0.500000,61.0,31.0,22.0,86.641375,41.810006,1020.4,1021.9,NaN,7.00000,NaN,19.6,0.0,NaN
1,2.1,15.7,NaN,5.700000,26.0,9.0,7.0,53.275715,76.598048,1023.7,1020.8,4.000000,1.00000,9.0,14.1,0.0,0.0
2,20.9,36.1,5.69496,NaN,39.0,31.0,17.0,15.471574,52.163254,1017.1,1014.9,4.167266,4.27812,24.2,33.9,0.0,0.0
4,8.2,20.5,3.20000,NaN,69.0,NaN,NaN,51.830889,51.770240,NaN,1001.2,7.000000,7.00000,NaN,17.6,0.0,0.6
5,9.7,24.1,5.69496,NaN,31.0,11.0,19.0,49.816899,58.470225,1022.9,NaN,4.167266,4.27812,17.8,23.2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,9.6,21.5,8.00000,12.000000,31.0,17.0,9.0,39.878487,57.490952,1024.5,1022.8,0.000000,6.00000,14.3,19.0,0.0,0.0
69996,11.6,31.4,5.69496,7.993227,39.0,7.0,NaN,NaN,70.743790,1019.1,1016.3,4.167266,4.27812,19.3,NaN,0.0,0.0
69997,22.2,34.5,NaN,7.600000,39.0,26.0,NaN,30.743840,64.412890,NaN,1008.5,7.000000,7.00000,25.2,NaN,0.0,0.0
69998,16.4,43.5,7.00000,12.700000,43.0,19.0,19.0,28.084916,63.565166,1014.2,1010.5,0.000000,4.27812,28.9,NaN,0.0,NaN


In [164]:
#針對非數值feature處理
#1.把nan去除後transform 算出median,2.transform整個feature並且把Nan值改成median 3.寫回df
def Non_Numeric_transform(feature):
    le = LabelEncoder()
    le.fit(df[feature])
    feature_Delete_NAN = le.transform(df[feature].dropna()) 
    print(feature_Delete_NAN[:20])
    feature_transform = le.transform(df[feature]) 
    #print("修改前:{}".format(feature_transform[:10]))
    for idx,nan in enumerate(pd.isna(df[feature])): #把nan的idx記錄下來
        if(nan==False):
            feature_transform[idx] = feature_transform[idx]
    #print("修改後:{}".format(feature_transform[:10]))
    df[feature] = feature_transform
    #print('{}有NAN? : {}'.format(feature,df[feature].isnull().values.any()))

In [165]:
Non_numeric_list = ['NightWindDir','DayWindDir','WindDir']
Numeric_list = [c for c in df.columns if c not in Non_numeric_list]


In [166]:
df  #乾淨的df

,TempLow,TempHigh,Steaming,Sun,WindSpeed,DayWindSpeed,NightWindSpeed,DayHumidity,NightHumidity,DayPressure,NightPressure,DayCloud,NightCloud,DayTemp,NightTemp,Weather,RISK_MM
0,19.5,NaN,5.69496,0.500000,61.0,31.0,22.0,86.641375,41.810006,1020.4,1021.9,NaN,7.00000,NaN,19.6,0.0,NaN
1,2.1,15.7,NaN,5.700000,26.0,9.0,7.0,53.275715,76.598048,1023.7,1020.8,4.000000,1.00000,9.0,14.1,0.0,0.0
2,20.9,36.1,5.69496,NaN,39.0,31.0,17.0,15.471574,52.163254,1017.1,1014.9,4.167266,4.27812,24.2,33.9,0.0,0.0
3,11.5,23.9,5.69496,7.993227,35.0,7.0,17.0,46.442398,67.664733,1010.2,NaN,4.167266,4.27812,13.8,21.8,1.0,0.2
4,8.2,20.5,3.20000,NaN,69.0,NaN,NaN,51.830889,51.770240,NaN,1001.2,7.000000,7.00000,NaN,17.6,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,9.6,21.5,8.00000,12.000000,31.0,17.0,9.0,39.878487,57.490952,1024.5,1022.8,0.000000,6.00000,14.3,19.0,0.0,0.0
69996,11.6,31.4,5.69496,7.993227,39.0,7.0,NaN,NaN,70.743790,1019.1,1016.3,4.167266,4.27812,19.3,NaN,0.0,0.0
69997,22.2,34.5,NaN,7.600000,39.0,26.0,NaN,30.743840,64.412890,NaN,1008.5,7.000000,7.00000,25.2,NaN,0.0,0.0
69998,16.4,43.5,7.00000,12.700000,43.0,19.0,19.0,28.084916,63.565166,1014.2,1010.5,0.000000,4.27812,28.9,NaN,0.0,NaN


In [167]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    df.drop(columns = ['Weather']).values[:train_end_idx, :],
    df['Weather'].values[:train_end_idx], test_size=0.5,shuffle=True)

X_test = df.drop(columns = ['Weather']).values[train_end_idx:, :]

In [168]:
X_val

array([[ 7.        , 14.1       ,  5.69496046, ...,  9.4       ,
        13.3       ,  0.        ],
       [15.6       , 21.2       ,  1.8       , ..., 18.7       ,
        20.2       ,  0.        ],
       [ 7.4       , 22.9       ,  5.69496046, ...,         nan,
        21.6       ,  0.        ],
       ...,
       [13.        ,         nan,  3.8       , ..., 16.3       ,
        14.3       ,  9.4       ],
       [11.7       ,         nan,  5.69496046, ..., 11.8       ,
        13.4       , 21.4       ],
       [24.8       ,         nan,  9.        , ..., 30.5       ,
        30.6       ,         nan]])

In [169]:
'''X_val = X_val[~np.isnan(X_val).any(axis=1)]
X_val.shape'''

'X_val = X_val[~np.isnan(X_val).any(axis=1)]\nX_val.shape'

In [170]:

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
print("train start")
X_train = imputer.fit_transform(X_train)
print("val start")
X_val = imputer.fit_transform(X_val)
print("test start")
X_test = imputer.fit_transform(X_test)

train start
val start
test start


In [171]:
from imblearn.over_sampling import SMOTE


sm = SMOTE( random_state=10) #10:0.33
X_train, y_train = sm.fit_resample(X_train, y_train)

In [172]:
from collections import Counter
print(Counter(y_train))

Counter({0.0: 15770, 1.0: 15770})


In [173]:
X_train

array([[13.6       , 20.2       ,  5.69496046, ..., 15.9       ,
        18.8       ,  0.        ],
       [18.        , 29.8       ,  7.4       , ..., 23.8       ,
        27.9       ,  1.13333333],
       [17.8       , 33.2       , 10.2       , ..., 25.4       ,
        31.1       ,  0.        ],
       ...,
       [-6.42541206, -3.26215662,  5.69496046, ..., -5.55027471,
        -3.92486265,  0.32321441],
       [ 8.67719127, 17.66607398,  5.80767192, ..., 11.72397555,
        13.45964717,  1.02222531],
       [14.06933474, 20.9224628 ,  5.69496046, ..., 17.18954654,
        18.8983317 ,  0.29958293]])

In [174]:
np.save('NotNML_X_train.npy',X_train)


In [ ]:
#這邊以上貼到preprocess.py(save那行不用)

In [175]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [176]:
X_train

array([[0.53562005, 0.47326733, 0.07887757, ..., 0.46943231, 0.47510373,
        0.        ],
       [0.65171504, 0.66336634, 0.10249307, ..., 0.6419214 , 0.66390041,
        0.00599647],
       [0.64643799, 0.73069307, 0.14127424, ..., 0.6768559 , 0.73029046,
        0.        ],
       ...,
       [0.00724506, 0.00867017, 0.07887757, ..., 0.00108571, 0.00363356,
        0.00171013],
       [0.40573064, 0.42309057, 0.08043867, ..., 0.37825274, 0.36430803,
        0.0054086 ],
       [0.54800356, 0.48757352, 0.07887757, ..., 0.49758835, 0.47714381,
        0.00158509]])

In [177]:
#scaler = MinMaxScaler(feature_range=(0, 1)).fit(X_val)
X_val = scaler.transform(X_val)

In [178]:
#scaler = MinMaxScaler(feature_range=(0, 1)).fit(X_test)
X_test = scaler.transform(X_test)

In [179]:
X_train

array([[0.53562005, 0.47326733, 0.07887757, ..., 0.46943231, 0.47510373,
        0.        ],
       [0.65171504, 0.66336634, 0.10249307, ..., 0.6419214 , 0.66390041,
        0.00599647],
       [0.64643799, 0.73069307, 0.14127424, ..., 0.6768559 , 0.73029046,
        0.        ],
       ...,
       [0.00724506, 0.00867017, 0.07887757, ..., 0.00108571, 0.00363356,
        0.00171013],
       [0.40573064, 0.42309057, 0.08043867, ..., 0.37825274, 0.36430803,
        0.0054086 ],
       [0.54800356, 0.48757352, 0.07887757, ..., 0.49758835, 0.47714381,
        0.00158509]])

In [180]:

'''from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import joblib
  
import numpy as np

np.save('X_train.npy',X_train)
np.save('X_val.npy',X_val)
np.save('y_train.npy',y_train)
np.save('y_val.npy',y_val)
np.save('X_test.npy',X_test)


#train tree model
clf1 = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
clf2 = RandomForestClassifier(n_estimators=50, criterion = 'gini')

clf3 = XGBClassifier(n_estimators=200, learning_rate= 0.3)

clf4 = MLPClassifier(random_state=1, max_iter=400)

clf5 = MLPClassifier(random_state=1, solver='sgd', max_iter=400)


Weather_model = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3), ('mlp1', clf4), ('mlp2', clf5)], voting='soft')


# Train Adaboost Classifer
Weather_model.fit(X_train, y_train)
#model = DecisionTreeClassifier()
#model.fit(X_train,y_train)

#save model
joblib.dump(Weather_model, 'Weather_model')'''

"from sklearn.tree import DecisionTreeClassifier\nfrom sklearn.ensemble import AdaBoostClassifier\nfrom sklearn.ensemble import RandomForestClassifier, VotingClassifier\nfrom sklearn.svm import SVC\nfrom xgboost import XGBClassifier\nfrom sklearn.neural_network import MLPClassifier\nfrom sklearn.neighbors import KNeighborsClassifier\nimport joblib\n  \nimport numpy as np\n\nnp.save('X_train.npy',X_train)\nnp.save('X_val.npy',X_val)\nnp.save('y_train.npy',y_train)\nnp.save('y_val.npy',y_val)\nnp.save('X_test.npy',X_test)\n\n\n#train tree model\nclf1 = AdaBoostClassifier(n_estimators=50,\n                         learning_rate=1)\nclf2 = RandomForestClassifier(n_estimators=50, criterion = 'gini')\n\nclf3 = XGBClassifier(n_estimators=200, learning_rate= 0.3)\n\nclf4 = MLPClassifier(random_state=1, max_iter=400)\n\nclf5 = MLPClassifier(random_state=1, solver='sgd', max_iter=400)\n\n\nWeather_model = VotingClassifier(estimators=[\n        ('lr', clf1), ('rf', clf2), ('gnb', clf3), ('mlp1', 

numpy.ndarray

In [181]:

from sklearn.metrics import accuracy_score, f1_score

X_train = np.load("X_train.npy")
X_val = np.load("X_val.npy")
y_train = np.load("y_train.npy")
y_val = np.load("y_val.npy")
X_test = np.load("X_test.npy")

model = joblib.load('Weather_model')
#predict
y_pred_decision = model.predict(X_val)
print('Accuracy: %f' % accuracy_score(y_val, y_pred_decision))
print('f1-score: %f' % f1_score(y_val, y_pred_decision))


Accuracy: 0.862100
f1-score: 0.432052


In [182]:
y_pred_decision[:10]

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 1.])

In [183]:
ans_pred = model.predict(X_test)
df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Weather'])
df_sap.to_csv('myAns.csv',  index_label = 'Id')

In [184]:
ans_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [185]:
len(X_train) #17578

31390

In [186]:
len( y_train)

31390

,Date,Loc,TempLow,TempHigh,Steaming,Sun,WindDir,WindSpeed,DayWindDir,NightWindDir,...,DayHumidity,NightHumidity,DayPressure,NightPressure,DayCloud,NightCloud,DayTemp,NightTemp,RISK_MM,Weather
31,2010/1/6,MountGambier,18.4,37.2,5.69496,7.993227,N,35.0,NNE,NE,...,25.314767,55.159208,1019.8,1017.3,4.167266,1.00000,24.6,36.0,0.0,0.0
62,2016/9/16,Sydney,20.1,33.6,2.80000,11.900000,NW,63.0,NNW,W,...,24.136189,61.115339,1004.8,1002.3,3.000000,1.00000,27.0,32.9,0.0,0.0
67,2010/7/17,Hobart,1.6,12.3,0.40000,1.000000,WNW,61.0,N,N,...,64.703081,51.389817,1011.4,1002.4,4.167266,4.27812,4.1,9.7,0.4,0.0
136,2011/12/27,CoffsHarbour,4.2,16.8,2.40000,9.900000,SSW,30.0,WSW,S,...,47.520234,52.893064,1028.1,1026.8,0.000000,1.00000,13.0,16.5,0.0,0.0
197,2016/6/23,Portland,9.7,15.3,1.80000,3.800000,WNW,65.0,WSW,WNW,...,69.674319,54.588586,1007.2,1003.9,8.000000,8.00000,11.9,14.4,8.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33713,2016/11/7,Tuggeranong,10.3,19.3,5.69496,7.993227,ESE,35.0,SSW,E,...,63.258968,53.926523,1015.9,1016.7,4.167266,4.27812,15.3,16.6,0.0,0.0
34047,2017/2/25,Witchcliffe,14.0,23.1,5.69496,7.993227,SSE,31.0,SSE,SSE,...,57.117167,60.291206,1016.3,1013.6,4.167266,4.27812,18.3,21.7,0.0,0.0
34305,2012/3/12,Nhil,8.3,14.9,5.69496,7.993227,WNW,20.0,NNW,W,...,60.251116,76.681905,1032.9,1030.2,4.167266,4.27812,9.1,13.7,0.0,0.0
34589,2015/1/19,Wollongong,15.4,25.2,5.69496,7.993227,NE,46.0,NNE,NE,...,78.250732,47.053021,1019.9,1015.2,3.000000,4.27812,20.1,19.8,0.0,0.0


,Date,Loc,TempLow,TempHigh,Steaming,Sun,WindDir,WindSpeed,DayWindDir,NightWindDir,...,DayHumidity,NightHumidity,DayPressure,NightPressure,DayCloud,NightCloud,DayTemp,NightTemp,RISK_MM,Weather
3819,2011/3/19,Adelaide,10.1,15.5,0.80000,7.993227,NNE,50.0,NW,NW,...,77.474915,69.081948,1007.4,1008.0,4.167266,4.27812,11.2,14.0,1.8,0.0
15232,2009/9/28,Adelaide,6.8,21.2,1.60000,10.400000,NNW,31.0,NE,NNW,...,44.819392,67.842552,1026.2,1021.6,4.167266,4.27812,14.0,20.7,0.0,0.0
18492,2011/1/24,Adelaide,14.8,28.9,4.60000,11.000000,ENE,33.0,ENE,NNE,...,25.533326,58.058745,1021.7,1018.8,4.167266,4.27812,19.3,26.9,0.0,0.0
19397,2016/2/1,Adelaide,11.7,16.0,5.69496,5.100000,WSW,30.0,SSW,WSW,...,69.318691,70.343203,1025.3,1023.7,4.167266,4.27812,12.6,14.9,0.0,0.0
21272,2014/4/22,Adelaide,6.5,20.1,1.60000,9.400000,WSW,17.0,NE,WNW,...,52.582089,59.132331,1022.3,1019.9,4.167266,4.27812,12.7,20.0,0.0,0.0
